In [12]:
import requests
import os
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import optimization_v1
import googlemaps
from dotenv import load_dotenv
import pytz
from datetime import datetime, timezone, timedelta
import pandas

load_dotenv()
proxy_address = "http://127.0.0.1:7890"
# Set the environment variables to specify the proxy
os.environ["HTTP_PROXY"] = proxy_address
os.environ["HTTPS_PROXY"] = proxy_address
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kaicheng/ProjectsFormal/wheelTrans/secrets/key.json"
os.environ["PROJECT_ID"] = "elderlyhometransportation"
googleMapKey = os.getenv("GOOGLE_MAP_API_KEY")

ROOT_PATH = os.path.dirname(os.path.abspath(''))
ROOT_PATH


'/Users/kaicheng/ProjectsFormal/wheelTrans/src'

In [13]:
def call_sync_api() -> None:
    request_file_name = "resources/request.json"
    fleet_routing_client = optimization_v1.FleetRoutingClient()

    with open(request_file_name) as f:
        fleet_routing_request = optimization_v1.OptimizeToursRequest.from_json(f.read())
        fleet_routing_request.parent = f"projects/elderlyhometransportation"
        fleet_routing_response = fleet_routing_client.optimize_tours(
            fleet_routing_request, timeout=10
        )
        return fleet_routing_response


def get_direction(origin, destination, waypoints):
    waypoints_str = "|".join(["optimize:true"] + waypoints)
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    google_map_api_key = os.getenv("GOOGLE_MAP_API_KEY")

    params = {
        "destination": destination,
        "origin": origin,
        "waypoints": waypoints_str,
        "key": google_map_api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


def get_geocoding(gmaps, address):
    result = gmaps.geocode(address)
    if not result or len(result) != 1:
        print(f"Failed to get geocoding for {address}")
        return None
    location = result[0]['geometry']['location']
    lat = location["lat"]
    lng = location["lng"]
    return [lat, lng]


origin = "8408 Garvey Ave. #101 Rosemead, CA 91770"
destination = "8408 Garvey Ave. #101 Rosemead, CA 91770"
address1 = "3843 Maxson Road #226 El Monte, CA 91732"
address2 = "119 Garcelon Ave Apt B Monterey Park, CA 91754"
waypoints = [origin, address1, address2, destination]
# get_direction(origin, destination, waypoints, destination)
# get_geocoding(origin)
# get_geocoding(address1)
# get_geocoding(address2)
# gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
# coordinates = []
# for location in waypoints:
#     coordinates.append(get_geocoding(gmaps, location))
# print(coordinates)
# response = call_sync_api()


In [ ]:
weekday = 'F' # 'M', 'T', 'W', 'R', 'F'

def handleInvalidAddress(df):
    print ("toDo")

df = pandas.read_excel("resources/TransportList.xlsx")
validDf = df[df['Address'].notna()]
handleInvalidAddress(df)
attendingAMdf = df[df[weekday].notna() & (df['Trans Method'].isin(['am', 'ampm']))]
attendingAMdfSubset = df.loc[:, ['Address','Trans Method', 'M', 'T', 'W', 'R', 'F', 'Notes']]
addresses = attendingAMdf['Address'].tolist()
#dev short
addresses = addresses[:10]

[attendingAMdfSubset,addresses, len(addresses)]

In [35]:
note = attendingAMdf[attendingAMdf['Notes'].notna()].at[0,'Notes']
note = note.strip()
for ind, row in attendingAMdf[attendingAMdf['Notes'].notna()].iterrows():
    print(row['Notes'])

8:00+
8:00+
8:00+
8:00+
8:00-
8:00-


In [15]:
len(addresses)

58

In [16]:
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
coordinates = []
validAddresses = []
invalidAddresses = []
for address in addresses:
    latlng = get_geocoding(gmaps, address)
    if latlng:
        coordinates.append(latlng)
        validAddresses.append(address)
    else:
        print(f"Failed to get geocoding for {address}")
        invalidAddresses.append(address)

coordinates

Failed to get geocoding for nan
Failed to get geocoding for nan


[[34.072393, -118.0102374],
 [34.0651666, -118.12373],
 [34.0651666, -118.12373],
 [34.0564491, -118.0874686],
 [34.0600187, -118.1244518],
 [34.0648659, -118.113868],
 [34.0564491, -118.0874686],
 [34.0113399, -118.1071289],
 [34.0113399, -118.1071289],
 [34.0474324, -118.0982999],
 [34.0474324, -118.0982999],
 [34.0673336, -118.1205658],
 [34.0255285, -118.1017612],
 [34.018299, -118.1129536],
 [34.018299, -118.1129536],
 [34.0643228, -118.097957],
 [34.0643228, -118.097957],
 [34.0793307, -118.1155368],
 [34.0793307, -118.1155368],
 [34.0950352, -118.1230117],
 [34.0979254, -118.1249617],
 [34.0564491, -118.0874686],
 [34.082718, -118.0706302],
 [34.0663829, -118.1221047],
 [34.0640168, -118.0607569],
 [34.0682325, -118.1184628],
 [34.072393, -118.0102374],
 [34.072393, -118.0102374],
 [34.0648659, -118.113868],
 [34.0498552, -118.0922655],
 [34.0495533, -118.1024123],
 [34.0616615, -118.1011805],
 [34.0877569, -118.0234481],
 [34.0762409, -118.0546959],
 [34.0587468, -118.0467618],

In [17]:
from resources.vehicles import vehicles
import json  

shipments = list(map(lambda latlng:
                     {
                         "loadDemands": {
                             "weight": {
                                 "amount": "1"
                             }
                         },
                         "pickups": [
                             {
                                 "arrivalLocation": {
                                     "latitude": latlng[0],
                                     "longitude": latlng[1]
                                 },
                                 "duration": "60s",
                                #  "timeWindows": [
                                #      {
                                #          "endTime": "1970-01-01T00:33:20Z",
                                #          "startTime": "1970-01-01T00:16:40Z"
                                #      }
                                #  ]
                             }
                         ]
                     }
                     , coordinates))
requestDict = {
    "parent": "projects/elderlyhometransportation",
    "model": {
        "shipments": shipments,
        "vehicles": vehicles,
        "global_start_time":"2024-03-08T7:15:00Z",
        "global_end_time":"2024-03-08T9:15:00Z",
        "global_duration_cost_per_hour": "60",
    },
    "searchMode":2
}
requestJson = json.dumps(requestDict)


In [18]:
fleet_routing_client = optimization_v1.FleetRoutingClient()
fleetOptimizationRequest = optimization_v1.OptimizeToursRequest.from_json(requestJson)

fleetOptimizationResponse = fleet_routing_client.optimize_tours(
    fleetOptimizationRequest, timeout=100,
)
fleetOptimizationResponse

routes {
  vehicle_start_time {
    seconds: 1709882100
  }
  vehicle_end_time {
    seconds: 1709885490
  }
  visits {
    shipment_index: 55
    is_pickup: true
    start_time {
      seconds: 1709882884
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
    }
    arrival_loads {
      type_: "weight"
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 51
    is_pickup: true
    start_time {
      seconds: 1709883836
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 231
    }
    arrival_loads {
      type_: "weight"
      value: 1
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 52
    is_pickup: true
    start_time {
      seconds: 1709883896
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 291
    }

In [19]:
optimizedRoutes = []
for vehicleRoute in fleetOptimizationResponse.routes:
    displayWaypoints=[]
    latlngWaypoints=[]
    for index, visit in enumerate(vehicleRoute.visits):
        shipmentId = visit.shipment_index
        displayWaypoints.append({
            'arrival': visit.start_time,
            'address': validAddresses[shipmentId]
        })
        latlngWaypoints.append(coordinates[shipmentId])
    optimizedRoutes.append({
        'vehicle': vehicleRoute.vehicle_index or 0,
        'order':latlngWaypoints,
        'displayWaypoints': displayWaypoints
        })
optimizedRoutes

[{'vehicle': 0,
  'order': [[34.1169439, -118.0834163],
   [34.1424524, -118.1448624],
   [34.1424524, -118.1448624],
   [34.0979254, -118.1249617]],
  'displayWaypoints': [{'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 28, 4, tzinfo=datetime.timezone.utc),
    'address': '6464 N Vista St San Gabriel, CA                     '},
   {'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 43, 56, tzinfo=datetime.timezone.utc),
    'address': '275 Cordova St Apt 606 Pasadena, CA'},
   {'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 44, 56, tzinfo=datetime.timezone.utc),
    'address': '275 Cordova St Apt 606 Pasadena, CA'},
   {'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 58, tzinfo=datetime.timezone.utc),
    'address': '111 N Chapel Ave Apt 805 Alhambra, CA 91801'}]},
 {'vehicle': 1,
  'order': [[34.0564491, -118.0874686],
   [34.0537935, -118.1174113],
   [34.053725, -118.1170457],
   [34.0537935, -118.1174113],
   [34.0762409, -118.0546959],
   [34.0877569, -118.0234481],
   [34

In [20]:
import polyline

def generate_static_map(origin, destination, directions, filename):

    # Extract route polyline from directions response
    route = directions[0]['overview_polyline']['points']
    decoded_route = polyline.decode(route)
    waypoints_polyline = '|'.join([f"{lat},{lng}" for lat, lng in waypoints])

    # Static Maps API request to generate map image with route and waypoints
    startingMarker = f'34.0623483,-118.0859541'
    static_map_url = f'https://maps.googleapis.com/maps/api/staticmap?size=1200x800&path=enc:{route}&markers=color:red|{waypoints_polyline}&markers=color:blue|{startingMarker}&key={googleMapKey}'
    print (static_map_url)
    # Download the map image
    map_image = requests.get(static_map_url)

    # Save the map image to a file
    with open('output/'+filename, 'wb') as f:
        f.write(map_image.content)


origin = (34.0623483, -118.0859541)  # Los Angeles, CA
destination = (34.0623483, -118.0859541)  # Los Angeles, CA
for route in optimizedRoutes:
    waypoints = route['order']
    print(waypoints)
    print(route['displayWaypoints'])
    directionResponse = gmaps.directions(origin, destination, waypoints=waypoints, mode="driving")
    vehicleInd = route['vehicle']
    generate_static_map(origin=origin, destination=destination, directions=directionResponse, filename=f'map_with_route{vehicleInd}.png' )



[[34.1169439, -118.0834163], [34.1424524, -118.1448624], [34.1424524, -118.1448624], [34.0979254, -118.1249617]]
[{'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 28, 4, tzinfo=datetime.timezone.utc), 'address': '6464 N Vista St San Gabriel, CA                     '}, {'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 43, 56, tzinfo=datetime.timezone.utc), 'address': '275 Cordova St Apt 606 Pasadena, CA'}, {'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 44, 56, tzinfo=datetime.timezone.utc), 'address': '275 Cordova St Apt 606 Pasadena, CA'}, {'arrival': DatetimeWithNanoseconds(2024, 3, 8, 7, 58, tzinfo=datetime.timezone.utc), 'address': '111 N Chapel Ave Apt 805 Alhambra, CA 91801'}]
https://maps.googleapis.com/maps/api/staticmap?size=1200x800&path=enc:ux{nEjvvoU}@?@hAAzAIPCl@@dAJX@x@@lC?jAGDAD?rB?pBe@?yA@yE@wUFsIBAGWAoC?yEF_NHeA@GBIH_A?o@?_@Ce@?iC@_RB_@?aBDaAEkB@qA?eC@uOBwb@NqEBwABMKeEBiGBc@?[JgD@sD?M?CCEC{ACK?S?UJwGBiCAmCBsC?{LBmGBsQFqKF}QHiE@gA@?OA{F?eE?iEAcGCsEAcOW?cC\gCXeBRsC

In [22]:
from PIL import Image, ImageDraw, ImageFont, ImageOps

def drawToImage(vehicleInd, waypoints):
    image = Image.open(ROOT_PATH+'/wheeltrans/output/'+f'map_with_route{vehicleInd}.png')
    image = image.convert("RGB")

    padded_image = ImageOps.expand(image, (500, 0), fill="white")

    draw = ImageDraw.Draw(padded_image)
    text ='\n'.join(waypoints)
    font = ImageFont.load_default()
    font_size = 20
    font = font.font_variant(size=font_size)
    text_position = (10, 10)
    text_color = (0, 0, 0)
    draw.text(text_position, text, fill=text_color, font=font)

    padded_image.save(f"output/image_with_text{vehicleInd}.png")

for route in optimizedRoutes:
    vehicleInd = route['vehicle']
    with open(f'output/waypoints{vehicleInd}', 'w') as f:
        addressWaypoints = list(map(lambda x: x['address'], route['displayWaypoints']))
        drawToImage(vehicleInd, addressWaypoints)